# Data Modeling with PostgreSQL

Jun Zhu

In [1]:
%load_ext autoreload
%autoreload 2

import pathlib
import psycopg2
import pandas as pd

from create_tables import init_database_and_tables
from sql_queries import (
    song_table_insert, song_table_check,
    artist_table_insert, artist_table_check,
    time_table_insert, time_table_check,
    user_table_insert, user_table_check,
    song_select, songplay_table_insert, songplay_table_check
)

psycopg2.__version__

'2.8.6 (dt dec pq3 ext lo64)'

In [2]:
init_database_and_tables()

In [3]:
conn = psycopg2.connect("host=127.0.0.1 dbname=sparkifydb port=5432 user=student password=student")

print("PostgreSQL server information")
print(conn.get_dsn_parameters(), "\n")

cur = conn.cursor()

PostgreSQL server information
{'user': 'student', 'dbname': 'sparkifydb', 'host': '127.0.0.1', 'port': '5432', 'tty': '', 'options': '', 'sslmode': 'prefer', 'sslcompression': '1', 'krbsrvname': 'postgres'} 



# Process `song_data`
In this first part, you'll perform ETL on the first dataset, `song_data`, to create the `songs` and `artists` dimensional tables.

Let's perform ETL on a single song file and load a single record into each table to start.
- Get a list of all song JSON files in `data/song_data`
- Select the first song in this list
- Read the song file and view the data

In [4]:
song_files = list(pathlib.Path("data/song_data").rglob("*.json"))

In [5]:
df = pd.read_json(song_files[0], typ='series')
df

num_songs                                         1
artist_id                        ARMAC4T1187FB3FA4C
artist_latitude                            40.82624
artist_longitude                          -74.47995
artist_location                   Morris Plains, NJ
artist_name               The Dillinger Escape Plan
song_id                          SOBBUGU12A8C13E95D
title               Setting Fire to Sleeping Giants
duration                                  207.77751
year                                           2004
dtype: object

## #1: `songs` Table
#### Extract Data for Songs Table
- Select columns for song ID, title, artist ID, year, and duration
- Use `df.values` to select just the values from the dataframe
- Index to select the first (only) record in the dataframe
- Convert the array to a list and set it to `song_data`

In [6]:
song_data = df[["song_id", "title", "artist_id", "year", "duration"]]
song_data

song_id                   SOBBUGU12A8C13E95D
title        Setting Fire to Sleeping Giants
artist_id                 ARMAC4T1187FB3FA4C
year                                    2004
duration                           207.77751
dtype: object

In [7]:
cur.execute(song_table_insert, song_data.values)
conn.commit()

In [8]:
cur.execute(song_table_check)

for row in cur.fetchall():
    print(row)

('SOBBUGU12A8C13E95D', 'Setting Fire to Sleeping Giants', 'ARMAC4T1187FB3FA4C', 2004, Decimal('207.77751'))


## #2: `artists` Table
#### Extract Data for Artists Table
- Select columns for artist ID, name, location, latitude, and longitude
- Use `df.values` to select just the values from the dataframe
- Index to select the first (only) record in the dataframe
- Convert the array to a list and set it to `artist_data`

In [9]:
artist_data = df[["artist_id", "artist_name", "artist_location", "artist_latitude", "artist_longitude"]]
artist_data

artist_id                  ARMAC4T1187FB3FA4C
artist_name         The Dillinger Escape Plan
artist_location             Morris Plains, NJ
artist_latitude                      40.82624
artist_longitude                    -74.47995
dtype: object

In [10]:
cur.execute(artist_table_insert, artist_data.values)
conn.commit()

In [11]:
cur.execute(artist_table_check)

for row in cur.fetchall():
    print(row)

('ARMAC4T1187FB3FA4C', 'The Dillinger Escape Plan', 'Morris Plains, NJ', Decimal('40.82624'), Decimal('-74.47995'))


# Process `log_data`
In this part, you'll perform ETL on the second dataset, `log_data`, to create the `time` and `users` dimensional tables, as well as the `songplays` fact table.

Let's perform ETL on a single log file and load a single record into each table.
- Get a list of all log JSON files in `data/log_data`
- Select the first log file in this list
- Read the log file and view the data

In [12]:
log_files = list(pathlib.Path("data/log_data").rglob("*.json"))

In [13]:
df = pd.read_json(log_files[0], lines=True)
df.head()

,artist,auth,firstName,gender,itemInSession,lastName,length,level,location,method,page,registration,sessionId,song,status,ts,userAgent,userId
0,Kenny G,Logged In,Chloe,F,53,Cuevas,256.57424,paid,"San Francisco-Oakland-Hayward, CA",PUT,NextSong,1.540941e+12,648,Everlasting,200,1542412944796,Mozilla/5.0 (Windows NT 5.1; rv:31.0) Gecko/20...,49
1,Randy Crawford,Logged In,Chloe,F,54,Cuevas,251.40200,paid,"San Francisco-Oakland-Hayward, CA",PUT,NextSong,1.540941e+12,648,Rio De Janeiro Blue (Album Version),200,1542413200796,Mozilla/5.0 (Windows NT 5.1; rv:31.0) Gecko/20...,49
2,Placebo,Logged In,Chloe,F,55,Cuevas,224.02567,paid,"San Francisco-Oakland-Hayward, CA",PUT,NextSong,1.540941e+12,648,Breathe Underwater,200,1542413451796,Mozilla/5.0 (Windows NT 5.1; rv:31.0) Gecko/20...,49
3,Poison The Well,Logged In,Chloe,F,56,Cuevas,184.60689,paid,"San Francisco-Oakland-Hayward, CA",PUT,NextSong,1.540941e+12,648,Riverside,200,1542413675796,Mozilla/5.0 (Windows NT 5.1; rv:31.0) Gecko/20...,49
4,Justin Bieber,Logged In,Chloe,F,57,Cuevas,196.88444,paid,"San Francisco-Oakland-Hayward, CA",PUT,NextSong,1.540941e+12,648,U Smile,200,1542413859796,Mozilla/5.0 (Windows NT 5.1; rv:31.0) Gecko/20...,49


## #3: `time` Table
#### Extract Data for Time Table
- Filter records by `NextSong` action
- Convert the `ts` timestamp column to datetime
  - Hint: the current timestamp is in milliseconds
- Extract the timestamp, hour, day, week of year, month, year, and weekday from the `ts` column and set `time_data` to a list containing these values in order
  - Hint: use pandas' [`dt` attribute](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.Series.dt.html) to access easily datetime-like properties.
- Specify labels for these columns and set to `column_labels`
- Create a dataframe, `time_df,` containing the time data for this file by combining `column_labels` and `time_data` into a dictionary and converting this into a dataframe

In [14]:
df = df[df['page'] == 'NextSong']
df.head()

,artist,auth,firstName,gender,itemInSession,lastName,length,level,location,method,page,registration,sessionId,song,status,ts,userAgent,userId
0,Kenny G,Logged In,Chloe,F,53,Cuevas,256.57424,paid,"San Francisco-Oakland-Hayward, CA",PUT,NextSong,1.540941e+12,648,Everlasting,200,1542412944796,Mozilla/5.0 (Windows NT 5.1; rv:31.0) Gecko/20...,49
1,Randy Crawford,Logged In,Chloe,F,54,Cuevas,251.40200,paid,"San Francisco-Oakland-Hayward, CA",PUT,NextSong,1.540941e+12,648,Rio De Janeiro Blue (Album Version),200,1542413200796,Mozilla/5.0 (Windows NT 5.1; rv:31.0) Gecko/20...,49
2,Placebo,Logged In,Chloe,F,55,Cuevas,224.02567,paid,"San Francisco-Oakland-Hayward, CA",PUT,NextSong,1.540941e+12,648,Breathe Underwater,200,1542413451796,Mozilla/5.0 (Windows NT 5.1; rv:31.0) Gecko/20...,49
3,Poison The Well,Logged In,Chloe,F,56,Cuevas,184.60689,paid,"San Francisco-Oakland-Hayward, CA",PUT,NextSong,1.540941e+12,648,Riverside,200,1542413675796,Mozilla/5.0 (Windows NT 5.1; rv:31.0) Gecko/20...,49
4,Justin Bieber,Logged In,Chloe,F,57,Cuevas,196.88444,paid,"San Francisco-Oakland-Hayward, CA",PUT,NextSong,1.540941e+12,648,U Smile,200,1542413859796,Mozilla/5.0 (Windows NT 5.1; rv:31.0) Gecko/20...,49


In [15]:
df['ts'] = pd.to_datetime(df['ts'], unit='ms')

t = df['ts']
t.head()

0   2018-11-17 00:02:24.796
1   2018-11-17 00:06:40.796
2   2018-11-17 00:10:51.796
3   2018-11-17 00:14:35.796
4   2018-11-17 00:17:39.796
Name: ts, dtype: datetime64[ns]

In [16]:
dt = t.dt
time_data = (t, dt.hour, dt.day, dt.isocalendar().week, dt.month, dt.year, dt.weekday)
column_labels = ("start_time", "hour", "day", "week", "month", "year", "weekday")

In [17]:
time_df = pd.DataFrame({k: v for k, v in zip(column_labels, time_data)})
time_df.head()

,start_time,hour,day,week,month,year,weekday
0,2018-11-17 00:02:24.796,0,17,46,11,2018,5
1,2018-11-17 00:06:40.796,0,17,46,11,2018,5
2,2018-11-17 00:10:51.796,0,17,46,11,2018,5
3,2018-11-17 00:14:35.796,0,17,46,11,2018,5
4,2018-11-17 00:17:39.796,0,17,46,11,2018,5


#### Insert Records into Time Table
Implement the `time_table_insert` query in `sql_queries.py` and run the cell below to insert records for the timestamps in this log file into the `time` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `time` table in the sparkify database.

In [18]:
for _, row in time_df.iterrows():
    cur.execute(time_table_insert, list(row))
    conn.commit()

In [19]:
cur.execute(time_table_check)

for row in cur.fetchall():
    print(row)

(datetime.datetime(2018, 11, 17, 0, 2, 24, 796000), 0, 17, 46, 11, 2018, 5)
(datetime.datetime(2018, 11, 17, 0, 6, 40, 796000), 0, 17, 46, 11, 2018, 5)
(datetime.datetime(2018, 11, 17, 0, 10, 51, 796000), 0, 17, 46, 11, 2018, 5)
(datetime.datetime(2018, 11, 17, 0, 14, 35, 796000), 0, 17, 46, 11, 2018, 5)
(datetime.datetime(2018, 11, 17, 0, 17, 39, 796000), 0, 17, 46, 11, 2018, 5)


## #4: `users` Table
#### Extract Data for Users Table
- Select columns for user ID, first name, last name, gender and level and set to `user_df`

In [20]:
user_df = df[['userId', 'firstName', 'lastName', 'gender', 'level']]
user_df = user_df.drop_duplicates('userId')
user_df.head()

,userId,firstName,lastName,gender,level
0,49,Chloe,Cuevas,F,paid
7,88,Mohammad,Rodriguez,M,free
23,10,Sylvie,Cruz,F,free
24,54,Kaleb,Cook,M,free
32,33,Bronson,Harris,M,free


#### Insert Records into Users Table
Implement the `user_table_insert` query in `sql_queries.py` and run the cell below to insert records for the users in this log file into the `users` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `users` table in the sparkify database.

In [21]:
for i, row in user_df.iterrows():
    cur.execute(user_table_insert, row)
    conn.commit()

In [22]:
cur.execute(user_table_check)

for row in cur.fetchall():
    print(row)

(49, 'Chloe', 'Cuevas', 'F', 'paid')
(88, 'Mohammad', 'Rodriguez', 'M', 'free')
(10, 'Sylvie', 'Cruz', 'F', 'free')
(54, 'Kaleb', 'Cook', 'M', 'free')
(33, 'Bronson', 'Harris', 'M', 'free')


## #5: `songplays` Table
#### Extract Data and Songplays Table
This one is a little more complicated since information from the songs table, artists table, and original log file are all needed for the `songplays` table. Since the log file does not specify an ID for either the song or the artist, you'll need to get the song ID and artist ID by querying the songs and artists tables to find matches based on song title, artist name, and song duration time.
- Implement the `song_select` query in `sql_queries.py` to find the song ID and artist ID based on the title, artist name, and duration of a song.
- Select the timestamp, user ID, level, song ID, artist ID, session ID, location, and user agent and set to `songplay_data`

#### Insert Records into Songplays Table
- Implement the `songplay_table_insert` query and run the cell below to insert records for the songplay actions in this log file into the `songplays` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `songplays` table in the sparkify database.

In [23]:
for _, row in df.iterrows():

    # get song_id and artist_id from the song and artist tables
    cur.execute(song_select, (row.song, row.artist, row.length))
    results = cur.fetchone()
    
    if results:
        song_id, artist_id = results
    else:
        song_id, artist_id = None, None

    # insert songplay record
    songplay_data = (
        row['ts'], row['userId'], row['level'], song_id, artist_id, 
        row['sessionId'], row['location'], row['userAgent']
    )

    cur.execute(songplay_table_insert, songplay_data)
    conn.commit()

In [24]:
cur.execute(songplay_table_check)

for row in cur.fetchall():
    print(row)

(1, datetime.datetime(2018, 11, 17, 0, 2, 24, 796000), 49, 'paid', None, None, 648, 'San Francisco-Oakland-Hayward, CA', 'Mozilla/5.0 (Windows NT 5.1; rv:31.0) Gecko/20100101 Firefox/31.0')
(2, datetime.datetime(2018, 11, 17, 0, 6, 40, 796000), 49, 'paid', None, None, 648, 'San Francisco-Oakland-Hayward, CA', 'Mozilla/5.0 (Windows NT 5.1; rv:31.0) Gecko/20100101 Firefox/31.0')
(3, datetime.datetime(2018, 11, 17, 0, 10, 51, 796000), 49, 'paid', None, None, 648, 'San Francisco-Oakland-Hayward, CA', 'Mozilla/5.0 (Windows NT 5.1; rv:31.0) Gecko/20100101 Firefox/31.0')
(4, datetime.datetime(2018, 11, 17, 0, 14, 35, 796000), 49, 'paid', None, None, 648, 'San Francisco-Oakland-Hayward, CA', 'Mozilla/5.0 (Windows NT 5.1; rv:31.0) Gecko/20100101 Firefox/31.0')
(5, datetime.datetime(2018, 11, 17, 0, 17, 39, 796000), 49, 'paid', None, None, 648, 'San Francisco-Oakland-Hayward, CA', 'Mozilla/5.0 (Windows NT 5.1; rv:31.0) Gecko/20100101 Firefox/31.0')


# Close Connection to Sparkify Database

In [25]:
conn.close()